## IMPORTING PACKAGES

In [1]:
import time
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import pandas as pd

In [2]:
X = pd.read_csv('data.csv', header=None)
y = pd.read_csv('label.csv', header=None)
y.columns = ['target']
num_clusters = y['target'].nunique()

combined = pd.concat([X, y], axis=1)
X = combined.iloc[:, :-1].values
y = combined.iloc[:, -1].values
num_clusters

10

In [3]:
class utils():
    def euclidean(A, B):
        return np.linalg.norm(A[:, np.newaxis] - B, axis=2)
    def cosine(A, B):
        dot_prod = np.dot(A, B.T)
        norm_A = np.linalg.norm(A, axis=1)[:, np.newaxis]
        norm_B = np.linalg.norm(B, axis=1)
        return 1 - dot_prod / (norm_A * norm_B)
    def jaccard(A, B):
        inter = np.minimum(A[:, np.newaxis], B).sum(axis=2)
        uni = np.maximum(A[:, np.newaxis], B).sum(axis=2)
        return 1 - inter / uni
    def dist_euclidean(x1, x2):
        return np.linalg.norm(x1 - x2)

    def dist_cosine(x1, x2):
        norm_x1 = np.linalg.norm(x1)
        norm_x2 = np.linalg.norm(x2)
        if norm_x1 == 0 or norm_x2 == 0:
            return 1
        return 1 - np.dot(x1, x2) / (norm_x1 * norm_x2)

    def dist_jaccard(x1, x2):
        intersection = np.sum(np.minimum(x1, x2))
        union = np.sum(np.maximum(x1, x2))
        if union == 0:
            return 1
        return 1 - intersection / union

    def euclidean1(x1, x2):
        return np.linalg.norm(x1 - x2, axis=-1)

    def cosine1(x1, x2):
        dot_product = np.dot(x1, x2)
        norm_x1 = np.linalg.norm(x1, axis=-1)
        norm_x2 = np.linalg.norm(x2)
        return 1 - dot_product / (norm_x1 * norm_x2)

    def jaccard1(x1, x2):
        intersection = np.minimum(x1, x2).sum(axis=-1)
        union = np.maximum(x1, x2).sum(axis=-1)
        return 1 - intersection / union

In [4]:

def kmeans(data_points, num_clusters, distance_function, max_iterations=200):
    num_points = data_points.shape[0]
    centers = data_points[np.random.choice(num_points, num_clusters, replace=False)]

    for iteration in range(max_iterations):
        dist_matrix = distance_function(data_points, centers)
        cluster_assignments = np.argmin(dist_matrix, axis=1)

        updated_centers = np.array([data_points[cluster_assignments == j].mean(axis=0) for j in range(num_clusters)])
        if np.all(centers == updated_centers):
            break
        centers = updated_centers

    sse = np.sum(np.min(distance_function(data_points, centers)**2, axis=1))
    return cluster_assignments, centers, sse


In [5]:
# Q1
distances = [utils.euclidean, utils.cosine, utils.jaccard]
names = ['Euclidean', 'Cosine', 'Jaccard']
results = {}
for metric, name in zip(distances, names):
    labels, centroids, sse = kmeans(X, num_clusters, metric)
    results[name] = {'SSE': sse}
print('sse -')
for name, result in results.items():
    print(f"{name}: {result['SSE']:.3f}")


sse -
Euclidean: 25484127692.115
Cosine: 681.841
Jaccard: 3661.362


In [6]:
# Q2

def kmeans(features, num_clusters, distance_function, max_iterations=200):
    num_samples, num_features = features.shape
    centroids = features[np.random.choice(num_samples, num_clusters, replace=False)]
    assignments = np.zeros(num_samples)

    for iter in range(max_iterations):
        for idx in range(num_samples):
            distances = [distance_function(features[idx], center) for center in centroids]
            assignments[idx] = np.argmin(distances)

        for cluster_idx in range(num_clusters):
            cluster_filter = assignments == cluster_idx
            if np.any(cluster_filter):
                centroids[cluster_idx] = np.mean(features[cluster_filter], axis=0)

    cluster_majority_labels = np.zeros(num_clusters)
    for cluster_idx in range(num_clusters):
        cluster_filter = assignments == cluster_idx
        if np.any(cluster_filter):
            dominant_label = np.argmax(np.bincount(labels[cluster_filter].astype(int)))
            cluster_majority_labels[cluster_idx] = dominant_label

    predictions = np.array([cluster_majority_labels[int(assignment)] for assignment in assignments])
    acc = np.sum(predictions == labels) / len(labels)

    return acc

dist_metrics = [utils.dist_euclidean, utils.dist_cosine, utils.dist_jaccard]
scores = {}

for distance, name in zip(dist_metrics, names):
    acc = kmeans(X, num_clusters, distance)
    print(f"{name} K-means Accuracy: {acc * 100:.3f}%")

Euclidean K-means Accuracy: 78.110%
Cosine K-means Accuracy: 77.630%
Jaccard K-means Accuracy: 74.800%


In [10]:
#Q3
def kmeans(data, k, metric, iterations=100, threshold=1e-4):
    num_samples, num_features = data.shape
    centroids = data[np.random.choice(num_samples, k, replace=False)]
    for iter in range(iterations):
        distances = np.array([metric(data, c) for c in centroids])
        cluster_assignments = np.argmin(distances, axis=0)
        new_centroids = np.array([data[cluster_assignments == j].mean(axis=0) for j in range(k)])
        if np.all(np.abs(new_centroids - centroids) < threshold):
            break
        centroids = new_centroids
    sse = np.sum([metric(data[i], centroids[cluster_assignments[i]])**2 for i in range(num_samples)])
    return cluster_assignments, centroids, iter + 1, sse
metrics = [utils.euclidean1, utils.cosine1, utils.jaccard1]
for metric, name in zip(metrics, names):
    st = time.time()
    cluster_labels, cluster_centroids, iterations, sse = kmeans(X, num_clusters, metric)
    en = time.time()
    print(f'{name}, SSE: {sse}, time: {en-st}, stop at: {iterations}')


Euclidean, SSE: 25323674868.06286, time: 33.68846940994263, stop at: 37
Cosine, SSE: 681.9138402294462, time: 64.65984344482422, stop at: 51
Jaccard, SSE: 3660.540464407733, time: 52.63206648826599, stop at: 49


In [11]:
def dist_euclidean(X, Y):
    return np.linalg.norm(X - Y, axis=1)

def sim_cosine(X, Y):
    dot_product = np.sum(X * Y, axis=1)
    norm_X = np.linalg.norm(X, axis=1)
    norm_Y = np.linalg.norm(Y)
    return 1 - dot_product / (norm_X * norm_Y)

def sim_jaccard(X, Y):
    intersection = np.sum(np.minimum(X, Y), axis=1)
    union = np.sum(np.maximum(X, Y), axis=1)
    return 1 - intersection / union

In [17]:

# Q4
def kmeans(data, k, dist_metric, stopping, iterations=100):
    num_samples, num_features = data.shape
    centr = data[np.random.choice(num_samples, k, replace=False)]
    prev_sse = float('inf')
    prev_centroids = np.copy(centr)

    stop_criterion = None

    for iteration in range(iterations):
        distances = np.array([dist_metric(data, c) for c in centr])

        if distances.ndim == 1:
            distances = distances.reshape(1, -1)
        elif distances.shape[0] != num_samples:
            distances = distances.T

        cluster_assignments = np.argmin(distances, axis=1)

        new_centr = []
        for j in range(k):
            cluster_points = data[cluster_assignments == j]
            if len(cluster_points) > 0:
                new_centr.append(cluster_points.mean(axis=0))
            else:
                new_centr.append(centr[j])
        updated_centr = np.array(new_centr)

        sse = np.sum([dist_metric(data[i:i+1], updated_centr[cluster_assignments[i]])**2 for i in range(num_samples)])

        if stopping == 'no_change' and np.all(np.abs(updated_centr - centr) < 1e-4):
            stop_criterion = 'no_change_in_centroids'
            break
        elif stopping == 'sse_increase' and sse > prev_sse:
            stop_criterion = 'sse_increase'
            break
        elif stopping == 'max_iterations' and iteration == iterations - 1:
            stop_criterion = 'max_iterations'
            break

        prev_sse = sse
        prev_centroids = np.copy(updated_centr)
        centr = updated_centr

    if stop_criterion is None:
        stop_criterion = 'max_iterations'

    return cluster_assignments, centr, iteration + 1, sse, stop_criterion

metrics = [dist_euclidean, sim_cosine, sim_jaccard]
names = ['Euclidean', 'Cosine', 'Jaccard']


for metric, name in zip(metrics, names):
    cluster_labels, cluster_centroids, num_iterations, sse, stop_criterion = kmeans(X, num_clusters, metric, stopping='no_change')
    print(f'{name} no change: {sse}')

    cluster_labels, cluster_centroids, num_iterations, sse, stop_criterion = kmeans(X, num_clusters, metric, stopping='sse_increase')
    print(f'{name} sse increase: {sse}')

    cluster_labels, cluster_centroids, num_iterations, sse, stop_criterion = kmeans(X, num_clusters, metric, stopping='max_iterations')
    print(f'{name} max iterations: {sse}')


Euclidean no change: 25423008529.697197
Euclidean sse increase: 25323825731.262337
Euclidean max iterations: 25536326668.147835
Cosine no change: 687.0408283771499
Cosine sse increase: 685.1686831854902
Cosine max iterations: 681.9577853410699
Jaccard no change: 3661.4583181338
Jaccard sse increase: 3661.4056538572645
Jaccard max iterations: 3660.711219349269
